# DRIVER DE PYTHON PARA CASSANDRA

# Instalación

### Prerrequisitos

El driver de Python para interactuar con Cassandra necesita CQL v3, Cassandra 2.1 o superior y Python en alguna de las siguientes versiones: 2.7, 3.4, 3.5, 3.6 y 3.7.

### Instalación a partir de pip

La forma más sencilla de instalar el driver es ejecutando el comando:

In [ ]:
!pip install cassandra-driver

El comando anterior emplea Cython para compilar varios componentes del driver de forma que posteriormente sea más rápida su ejecución. Tiene el incoveniente que la instalación del driver tomará varios minutos para completarse.

# Conexión al Cluster

Para conectarnos a un cluster de Cassandra empleando este driver el primer paso será importar las librerías necesarias:

In [ ]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

El paso siguiente será instanciar un objeto de la clase Cluster. Podemos no especificar ningún argumento con lo que dicha instancia serviría para realizar una conexión posterior con el servicio de Cassandra que se esté ejecutando en localhost:

In [ ]:
cluster_a = Cluster()

En caso de que se necesite autenticación para acceder al cluster habrá que instanciar alguna subclase de AuthProvider como PlainTextAuthProvider de la siguiente forma:

In [ ]:
cluster_auth = Cluster(auth_provider=PlainTextAuthProvider(username='cassandra', password='cassandra'))

Pero también podemos pasar a dicha llamada una lista con IPs de nodos del cluster de forma que en el momento que contacte con alguna de ellas, automáticamente le permitirá descubrir el resto de nodos. Además es posible especificar opciones como el puerto. Hay que señalar que los puertos por defecto para la comunicación del cluster son el 7000, el 7001 en caso de que SSL esté activo y 9042 para native protocol clients (define el formato de los mensajes binarios entre el driver y el cluster).

In [ ]:
# cluster_b = Cluster([192.168.0.27, 192.168.0.32, 192.168.0.34], port=9042)

A continuación, se empleará el objeto cluster para crear una sesión llamando al método connect que acepta como argumento opcional el keyspace con el que se desa realizar la conexión.

In [ ]:
sess = cluster_auth.connect('clientes')

También se puede cambiar a un keyspace determinado mediante la sentencia:

In [ ]:
sess.set_keyspace('clientes')

O ejecutando el comando de CQL USE a través del driver de Python:

In [ ]:
sess.execute('USE clientes')

# Ejecución de consultas

Acabamos de ver un anticipo de cómo ejecutar consultas empleando el driver, es decir, invocando al método execute().

In [ ]:
filas = sess.execute("SELECT * FROM cuenta_zero123")

El resultado se devolverá en forma de tupla.

In [ ]:
for fila in filas:
    print("\nNueva línea")
    for e in fila:
        print(e) 

In [ ]:
filas = sess.execute("SELECT * FROM cuenta_zero123")
for fila in filas:
    print(fila)

Las tuplas que devuelve la ejecución de una consulta contienen los nombres de los campos de las filas.

In [ ]:
filas = sess.execute("SELECT nombre, email, mov_dt, saldo FROM cuenta_zero123")
for fila in filas:
    print("Cliente: " + unicode(fila.nombre) + " con email: " + str(fila.email))

Ejemplo en el que se hace referencia al nombre de las columnas tras ejecutar una consulta:

In [ ]:
filas = sess.execute("SELECT nombre, email, mov_dt, saldo FROM cuenta_zero123")
for (nombre, email, _, _) in filas:
    print(nombre, email)

### Paso de parámetros

Los nombres de keyspaces, tablas y columnas deben escribirse explícitamente pero se pueden pasar como parámetro valores de columnas de la consulta / inserción / actualización / borrado teniendo en cuenta que el tipo de todos los argumentos habrá que definirlo como %s y se debe emplear una secuencia (tupla, lista) para especificar los valores de dichos argumentos aunque haya sólo 1. Si se emplea una tupla y hay sólo 1 argumento la forma de indicar su valor será (arg,). 

Ejemplo en el que se pasan como parámetros los valores de 2 columnas en cláusula WHERE:

In [ ]:
filas = sess.execute("SELECT * FROM cuenta_zero123 WHERE nombre = %s and email = %s", ("Alberto Martín", "am@gmail.com"))

In [ ]:
for fila in filas:
    print(fila)

Ejemplo en el que se pasa como parámetro el valor de 1 columna a la hora de realizar una inserción:

In [ ]:
sess.execute("INSERT INTO cuenta_zero123 (nombre, email, mov_dt, saldo) VALUES (%s, 'rb@gmail.com', '2019-09-21', 2034.23)", ("Ruth Bosch",))

Se comprueba la inserción realizada pasando como parámetros los valores de 2 columnas en cláusula WHERE empleando una tupla:

In [ ]:
filas = sess.execute("SELECT * FROM cuenta_zero123 WHERE nombre = %s and email = %s", ("Ruth Bosch", "rb@gmail.com"))

In [ ]:
for fila in filas:
    print(fila)

Se pueden emplear placeholders con nombre de la siguiente forma:

In [ ]:
sess.execute("INSERT INTO cuenta_zero123 (nombre, email, mov_dt, saldo) VALUES (%(nombre)s, %(email)s, %(mov_dt)s, %(saldo)s)", {"nombre": "Francisco Serrano", "email": "frs@gmail.com", "mov_dt": "2019-09-24", "saldo": 6040.23})

De igual manera, se verifica la inserción realizada pasando como parámetros los valores de 2 columnas en la cláusula WHERE empleando una lista:

In [ ]:
filas = sess.execute("SELECT * FROM cuenta_zero123 WHERE nombre = %s and email = %s", ["Francisco Serrano", "frs@gmail.com"])

In [ ]:
for fila in filas:
    print(fila)

# Consultas asíncronas

Las consultas realizadas en el apartado anterior eran síncronas, es decir, el driver lanza la consulta y hasta que dicha consulta no finaliza y se han obtenido las filas, el driver no puede continuar con la ejecución de siguientes cláusulas.

Las consultas asíncronas se realizan empleando el método <b>execute_async()</b> y devuelven inmediatamente un objeto <b>ResponseFuture</b>. Dicho objeto dispone de una serie de métodos como <b>result()</b>, <b>add_callback()</b>, <b>add_callbacks()</b> o <b>add_errback()</b>.

Si desde el objeto ResponseFuture se invoca al método <b>result()</b> se bloqueará la ejecución del driver hasta que la consulta finalice devolviendo su resultado o hasta que se lance una excepción en caso de producirse un error.

In [ ]:
from cassandra import ReadTimeout

res_fut = sess.execute_async("SELECT * FROM cuenta_zero123 WHERE nombre = %s AND email = %s", ["Francisco Serrano", "frs@gmail.com"], timeout=20)

try:
    filas = res_fut.result()
    for fila in filas:
        print(fila)
except ReadTimeout:
    print("La consulta ha execedido el tiempo permitido") # sess = cluster.connect('clientes', default_timeout=10) | sess.execute_async("SELECT ...", timeout=20) | sess.execute("SELECT ...", timeout=20)

A continuación se mostrará un ejemplo en el que se ejecutarán de forma asíncrona 3 consultas guardando en una lista 3 objetos ResponseFuture y posteriormente se obtendrán sus resultandos invocando al método bloqueante result() sobre cada uno de ellos:

In [ ]:
nombres = ["Francisco Serrano", "Ruth Bosch", "Alberto Martín"]
emails = ["frs@gmail.com", "rb@gmail.com", "am@gmail.com"]
res_futs = []

for nombre, email in zip(nombres, emails):
    res_futs.append(sess.execute_async("SELECT * FROM cuenta_zero123 WHERE nombre = %s AND email = %s", [nombre, email]))
    
for res_fut in res_futs:
    filas = res_fut.result()
    for fila in filas:
        print(fila)

El método <b>add_callback()</b> del objeto ResponseFuture perimite añadir una función de callback que será invocada cuando se obtengan los resultados. Dicha función recibirá como parámetro los resultados. Si se llama a add_callback(fn, \*args, \*\*kwargs) entonces \*args se pasará como una secuencia de longitud variable de argumentos posicionales y \*\*kwargs se pasará como un diccionario de pares clave o nombre de variable-valor. Si se produce algún error durante la ejecución no se llamará a ningún callback. Para esos casos habrá que utilizar add_callbacks() o add_errback(). El callback se ejecuta en un thread de IO y ningún otro thread será procesado hasta que el callback finalice. Otro punto importante a tener en cuenta consiste en que si se produce alguna excepción en el callback, ésta será ignorada por lo que es aconsejable tratar las excepciones que puedan ocurrir.

Vamos a ver un ejemplo de uso de <b>execute_async() con el método add_callback(fn, \*args, \*\*kwargs)</b>:

In [ ]:
def my_cb(filas, num, msg):
    print(msg)
    print(num)
    for fila in filas:
        try:
            print("El saldo de " + str(fila.nombre) + " es " + str(fila.saldo))
        except:
            print("Error al mostrar el nombre del cliente por la presencia de caracteres no interpretables.")

res_fut = sess.execute_async("SELECT * FROM cuenta_zero123")
filas = res_fut.add_callback(my_cb, 3, "Entrando al callback")

In [ ]:
def my_cb(filas, num, msg):
    print(msg)
    print(num)
    for fila in filas:
        try:
            print("El saldo de " + unicode(fila.nombre) + " es " + str(fila.saldo))
        except:
            print("Error al mostrar el nombre del cliente por la presencia de caracteres no interpretables.")

res_fut = sess.execute_async("SELECT * FROM cuenta_zero123")
filas = res_fut.add_callback(my_cb, 3, "Entrando al callback")

Ejemplo de uso del método <b>execute_async() con add_errback(fn, \*args, \*\*kwargs)</b>:

In [ ]:
def my_errcb(msg, msg2):
    print("* Mensaje devuelto por la excepción: " + str(msg))
    print("\n* Parámetro adicional pasado al add_errback: " + str(msg2))
    
res_fut = sess.execute_async("SELECT * FROM cuenta_zero123 WHERE saldo > 5000")
filas = res_fut.add_errback(my_errcb, "No se ha especificado una partition key válida a la hora de realizar el filtrado WHERE")

Ejemplo de uso del método <b>execute_async() con add_callbacks(callback_fn, errback_fn, \*callback_args=(), \*\*callback_kwargs=None, \*errback_args=())</b>: 

In [ ]:
def mycb(filas, num, msg, dic):
    print(msg)
    print(num)
    print(dic.keys())
    print(dic.values())
    for fila in filas:
        try:
            print("El saldo de " + unicode(fila.nombre) + " es " + str(fila.saldo))
        except:
            print("Error al mostrar el nombre del cliente por la presencia de caracteres no interpretables.")

def my_errcb(msg, msg2):
    print("* Mensaje devuelto por la excepción: " + str(msg))
    print("\n* Mensaje pasado al add_errback: " + str(msg2))
    #print("\n* Valor del diccionario pasado al add_errback: " + str(msg2.value()))
    
res_fut = sess.execute_async("SELECT * FROM cuenta_zero123 WHERE saldo > 5000")
filas = res_fut.add_callbacks(callback=mycb, errback=my_errcb, callback_args=(1, "Entrando al callback"), callback_kwargs={'k': 2}, errback_args=("No se ha especificado una partition key válida a la hora de realizar el filtrado WHERE",))

In [ ]:
def mycb(filas, num, msg, k):
    print(msg)
    print(num)
    print(k.items())

    for fila in filas:
        try:
            print("El saldo de " + unicode(fila.nombre) + " es " + str(fila.saldo))
        except:
            print("Error al mostrar el nombre del cliente por la presencia de caracteres no interpretables.")

def my_errcb(msg, msg2):
    print("* Mensaje devuelto por la excepción: " + str(msg))
    print("\n* Clave del diccionario pasado al add_errback: " + str(msg2))
    #print("\n* Valor del diccionario pasado al add_errback: " + str(msg2.value()))
    
res_fut = sess.execute_async("SELECT * FROM cuenta_zero123 WHERE nombre=%(nombre)s AND email=%(email)s", {"nombre": "Francisco Serrano", "email": "frs@gmail.com"})
filas = res_fut.add_callbacks(callback=mycb, errback=my_errcb, callback_args=(1, "Entrando al callback"), callback_kwargs={'k': 2}, errback_args=("No se ha especificado una partition key válida a la hora de realizar el filtrado WHERE",))

# Prepared Statements

Son consultas que Cassandra preprocesa a modo de placeholder de forma que podrán ser ejecutadas posteriormente dando valores a sus parámetros. Esto permite reducir el tráfico de red y la utilización de procesador ya que no hay que volver a parsear la consulta en caso de que vaya a ser ejecutada varias veces pasándole diferentes valores a sus parámetros. 

La forma de crear una prepared statement es mediante <b>sess.prepare("SELECT ...FROM ... WHERE col=?")</b>. Esa llamada devolverá un objeto <b>PreparedStatement</b> de forma que el driver se encarga de preparar ("parsear") esa consulta en todos los nodos y de re-prepararla en nodos nuevos que se añadan posteriormente. 

In [ ]:
nombres = ["Francisco Serrano", "Ruth Bosch", "Alberto Martín"]
emails = ["frs@gmail.com", "rb@gmail.com", "am@gmail.com"]

prep = sess.prepare("SELECT * FROM cuenta_zero123 WHERE nombre=? AND email=?")

for nombre, email in zip(nombres, emails):
    filas = sess.execute(prep, [nombre, email])
    for fila in filas:
        print(fila)